# Implementación técnica de Google Analytics con Google Tag Manager

<img src="img/keywords.png">

## Google Analytics: protocolo de medición

- Un conjunto de reglas estándar para recopilar y enviar visitas desde cualquier dispositivo conectado a Internet a Analytics.
- El protocolo de medición de Google Analytics permite a los programadores realizar **solicitudes HTTP** para enviar datos de interacción de usuario sin procesar, directamente a los servidores de Google Analytics (Más pro)

Enlaces:
- [Hit Builder](https://ga-dev-tools.appspot.com/hit-builder/)
- [Descripción general del Protocolo de medición](https://developers.google.com/analytics/devguides/collection/protocol/v1/)
- [Validar hits: Protocolo de medición](https://developers.google.com/analytics/devguides/collection/protocol/v1/validating-hits)

### ¿A dónde envío los datos?

- Google Analytics dispone de un *extremo* o *endpoint* para recibir la información mediante hits.

```
POST /collect HTTP/1.1
Host: www.google-analytics.com
```

- A dicho *endpoint* debemos enviar la información mínima requerida y en el formato especificado para que el hit sea validado correctamente por el servidor de Analytics.

**Tipos de hits:**

- Seguimiento de páginas: `t=pageview`
- Seguimiento de eventos: `t=event`
- Seguimiento de transacciones (No EEC): `t=transaction`
- Seguimiento de artículos (No EEC): `t=item`
- Seguimiento de interacciones sociales: `t=social`
- Seguimiento de interacciones excepciones o errores de javascript: `t=exception`
- Seguimiento de tiempos de usuario: `t=timing`
- Seguimiento en la aplicación o en la pantalla (Mobile): `t=screenview` 

> No existe un hit específico para el envío de la información de Comercio Electrónico Mejorado. La información se anexa a cualquier otro hit (excepto el de transacciones `t=transaction` y el de artículos `t=item`)

### Como se envía un hit

Lo más habitual en hacerlo mediante el uso de tags.

> Tag: pequeño trozo de código, de apenas unas líneas, que es colocado en una o varias páginas web con el fin de realizar la acción para la cual ha sido diseñado.


### Envío con Javascript de un hit de página vista a Analytics haciendo uso del protocolo de medición.

- No estamos empleando ninguna librería Javascript.
- Empleamos funciones Javascript disponibles en todos los navegadores.
- Configuramos el hit de forma manual

> Ejecuta el siguiente código en la consola. ¿Cuál es la respuesta?

In [17]:
%%javascript 
var xhr = new XMLHttpRequest();
xhr.open("POST", 'http://www.google-analytics.com/collect?v=1&tid=UA-62172390-1&cid=1234567&t=pageview&dp=%2Fhome', true);
xhr.send(null);

<IPython.core.display.Javascript object>

- Google Analytics dispone de un servidor de validación del protocolo de medición. Su *endpoint* es:

```
POST /debug/collect HTTP/1.1
Host: www.google-analytics.com
```

> Vamos a enviar el siguiente hit y auditarlo en la consola mirando su respuesta:

In [ ]:
%%javascript 
var xhr = new XMLHttpRequest();
xhr.open("POST", 'http://www.google-analytics.com/debug/collect?v=1&tid=UA-62172390-1&cid=1234567&t=pageview', true);
xhr.send(null);

Podemos obtener la respuesta que nos envía el servidor de Analytics para su procesamiento

In [18]:
%%javascript
var xhr = new XMLHttpRequest();
xhr.onreadystatechange = function() {
    if (xhr.readyState == XMLHttpRequest.DONE) {
        console.log(xhr.responseText);
    }
}
xhr.open("POST", 'http://www.google-analytics.com/debug/collect?v=1&tid=UA-62172390-1&cid=1234567&t=pageview', true);
xhr.send(null);

<IPython.core.display.Javascript object>

### analytics.js: La librería JavaScript empleada por Google Analytics para medir cómo interactúan los usuarios con un sitio web

Esto es una visión minimalista de lo que realiza la librería de Google Analytics al insertarla en un página web:

```
<script>
  (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
  (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
  m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
  })(window,document,'script','https://www.google-analytics.com/analytics.js','ga');
  ga('create', 'UA-62172390-1', 'auto');
  ga('send', 'pageview');
</script>

```


### Vamos a crear nuestro propio tag de medición básica

Haciendo uso de Javascript es posible acceder a toda la información que nos proporciona el navegador del usuario:

In [19]:
%%javascript
// Dominio del usuario
console.log(window.location.hostname)

<IPython.core.display.Javascript object>

In [20]:
%%javascript
// Página por la que navega el usuario
console.log(window.location.pathname)

<IPython.core.display.Javascript object>

In [21]:
%%javascript
// Título de la página
console.log(document.title)

<IPython.core.display.Javascript object>

In [22]:
%%javascript
// Navegador del usuario
console.log(navigator.userAgent)

<IPython.core.display.Javascript object>

Cuando enviamos hits a Analytics a través del protocolo de medición no es necesario especificar parámetros como el hostname o nombre del dominio desde el cual se envía el hit, navegador del usuario, su dirección IP o ubicación geográfica. Dichos parámetros viajan siempre en las [cabeceras http](https://en.wikipedia.org/wiki/List_of_HTTP_header_fields) de cualquier petición web.

Todas la demás variables debemos incluirlas.


### Cookies

analytics.js usa dos cookies propias para:

- Distinguir a los usuarios únicos: Cookie `_ga` (Duración 2 años)
- Limitar el porcentaje de solicitudes: Cookie: `_gat` (Duración 10 minutos)


Para que Analytics determine que dos hits diferentes pertenecen al mismo usuario, con cada hit se debe enviar un identificador único asociado a ese usuario.

analytics.js utiliza el campo `clientId`, una cadena única generada aleatoriamente que se almacena en las cookies del navegador.

> Crear una cookie con Javascript es muy sencillo

In [23]:
%%javascript
var cookieName = '_ga';
var clientId = '76c24efd-ec42-492a-92df-c62cfd4540a3' // cadena única generada aleatoriamente
var date = new Date();
date.setTime(date.getTime() + (365 * 2 * 24 * 60 * 60 * 1000));
var expires = "; expires=" + date.toGMTString();
// Escribimos la cookie:
document.cookie = cookieName + "=" + clientId + expires + "; path=/";

<IPython.core.display.Javascript object>

In [24]:
%%javascript
// Veamos todas las cookies del dominio en el que estamos:
console.log(document.cookie)

<IPython.core.display.Javascript object>

> Y leerlas también

In [25]:
%%javascript
function getCookie(name) {
  var value = "; " + document.cookie;
  var parts = value.split("; " + name + "=");
  if (parts.length == 2) return parts.pop().split(";").shift();
}
console.log(getCookie('_ga'))

<IPython.core.display.Javascript object>

### Apliquemos lo aprendido a nuestro tag:

In [26]:
%%javascript

// Variables
function getCookie(name) {
  var value = "; " + document.cookie;
  var parts = value.split("; " + name + "=");
  if (parts.length == 2) return parts.pop().split(";").shift();
}

var ua = 'UA-62172390-1';
var clientId = getCookie('_ga');
var tipoHit = 'pageview';
var pagePath = window.location.pathname;
var pageTitle = document.title;

var xhr = new XMLHttpRequest();
xhr.onreadystatechange = function() {
    if (xhr.readyState == XMLHttpRequest.DONE) {
        console.log(xhr.responseText);
    }
}
xhr.open("POST", 'http://www.google-analytics.com/collect?v=1&tid=' + ua + '&cid=' + clientId + '&t=' + tipoHit + '&dp=' + pagePath + '&dt=' + pageTitle + '', true);
xhr.send('');

<IPython.core.display.Javascript object>

## Eventos

- Eventos HTML
- Eventos en Analytics

### Enventos HTML

Un evento es algo que hace la página web o el usuario. 

> [Debemos conocerlos para saber qué podemos medir](http://www.w3schools.com/tags/ref_eventattributes.asp)

Algunos ejemplos de eventos HTML:

- La página ha terminado de cargarse: `ònload`
- Se ha producido un cambio en un campo de un formulario. Ejemplos: `oninput`, `onblur`,  `onchange`, etc.
- Se ha interactuado con un botón. Ejemplos `onclick`, `ondblclick`, `onmouseover`, etc.

Mediante Javascript pordemos ejecutar código cuando se detecta un evento.

**La forma básica (no recomendable pero válida):**

In [27]:
%%html
<!DOCTYPE html>
<html>
<title>Carrito</title>
<head>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css">
  <script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.4/jquery.min.js"></script>
  <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js"></script>
</head>
<body>

<div class="container">
  <h2>El arte de medir</h2>
  <img src="https://images-na.ssl-images-amazon.com/images/I/51k86SlEn4L._AC_UL115_.jpg" />
  <p>Este libro es una introducción a la análitica web, explica los conceptos necesarios para poder sacarle el máximo partido a los datos.</p>
  <button type="button" class="btn btn-danger btn-lg" id="addToCart" onclick="console.log('Click en el botón')"><span class="glyphicon glyphicon-shopping-cart"></span> Añadir al carrito</button>
</div>

</body>
</html>

---
Mediante escuchadores de eventos o listeners. En este caso usando [jquery](https://jquery.com/).

In [ ]:
%%javascript
$('#addToCart').mouseover(function(){
    console.log('Estoy encima del botón que pone ' + $(this).text());
});

In [ ]:
%%javascript
// Quitamos el listener
$('#addToCart').unbind('mouseover');


### Eventos en Analytics

Los eventos son interacciones del usuario con contenido cuyo seguimiento se puede realizar independientemente a partir de una página web o una carga de pantalla. Las descargas, los clics en anuncios para móviles, los gadgets, los elementos Flash, los elementos insertados AJAX y las reproducciones de vídeo son todos ejemplos de acciones que podemos medir con eventos. [Más info](https://support.google.com/analytics/answer/1033068)

- Para enviar datos de eventos los hits deben ser del tipo `event`
- Variables específicas de los eventos: 
    - `ec`: Categoría del evento (Obligatorio)
    - `ea`: Acción del evento (Obligatorio)
    - `el`: Etiqueta del evento (Recomendable)
    - `ev`: Valor del evento
    

In [28]:
%%javascript

// Variables
function getCookie(name) {
  var value = "; " + document.cookie;
  var parts = value.split("; " + name + "=");
  if (parts.length == 2) return parts.pop().split(";").shift();
}

var ua = 'UA-62172390-1';
var clientId = getCookie('_ga');
var tipoHit = 'event';
var pagePath = window.location.pathname;
var pageTitle = document.title;

var xhr = new XMLHttpRequest();
xhr.onreadystatechange = function() {
    if (xhr.readyState == XMLHttpRequest.DONE) {
        console.log(xhr.responseText);
    }
}

$('#addToCart').click(function(){
    var eventCategory = 'Interacción';
    var eventAction = 'Carrito';
    var eventLabel = $(this).text();
    xhr.open("POST", 'http://www.google-analytics.com/collect?v=1&tid=' + ua + '&cid=' + clientId + '&t=' + tipoHit + '&dp=' + pagePath + '&dt=' + pageTitle + '&ec=' + eventCategory + '&ea=' + eventAction + '&el=' + eventLabel + '  ' , true);
    xhr.send('');
});

<IPython.core.display.Javascript object>

In [ ]:
%%javascript
// Quitamos el listener
$('#addToCart').unbind('click');

> ¿Podemo organizalo todo mejor?

## El dataLayer

Es un objeto javascript en formato JSON (`'clave' : 'valor'`) que contiene información, tanto de  eventos como de variables. 

Su principal objetivo es crear una capa de datos que facilite el acceso a los datos a herramientas como **Google Tag Manager**.

In [29]:
%%javascript

var dataLayer = [{
    'pageLanguage': 'es_ES',
    'pageTemplate': 'ProductDetail',
    'userLoggedIn': false,
    // Comecio electrónico mejorado
    'ecommerce': {
        'detail': {
          'actionField': {'list': 'Top ventas'}, 
          'products': [{
            'name': 'El Arte de Medir', 
            'id': '12345',
            'price': '15.25',
            'category': 'Web analytics',
            'stock': 3,
            'edition': 2
           }]
         }
   }
}]

console.log(dataLayer)

<IPython.core.display.Javascript object>

[Enhanced Ecommerce (UA) Developer Guide](https://developers.google.com/analytics/devguides/collection/analyticsjs/enhanced-ecommerce)

### ¿Cómo agregar nueva información al dataLayer?

In [30]:
%%javascript

var dataLayer = [{
    'pageLanguage': 'es_ES',
    'pageTemplate': 'ProductDetail',
    'userLoggedIn': false,
    'ecommerce': {
        'detail': {
          'actionField': {'list': 'Top ventas'}, 
          'products': [{
            'name': 'El Arte de Medir', 
            'id': '12345',
            'price': '15.25',
            'category': 'Web analytics',
            'stock': 3,
            'edition': 2
           }]
         }
   }
}]

dataLayer.push({
    'event': 'ecommerce',
    'eventCategory': 'Interacción',
    'eventAction': 'Carrito',
    'ecommerce': {
        'currencyCode': 'EUR',
        'add': {   
          'products': [{ 
            'name': 'El Arte de Medir', 
            'id': '12345',
            'price': '15.25',
            'category': 'Web analytics',
            'stock': 3,
            'edition': 2,
            'quantity': 1
           }]
        }
    }
});

console.log(dataLayer)



<IPython.core.display.Javascript object>

Por sencillez vamos a suponer vamos a suponer:

In [ ]:
%%javascript

var dataLayer = [{
    'language': 'es_ES',
    'pageTemplate': 'ProductDetail'
}]

console.log(dataLayer)

In [ ]:
%%javascript

// VARIABLES
function getCookie(name) {
  var value = "; " + document.cookie;
  var parts = value.split("; " + name + "=");
  if (parts.length == 2) return parts.pop().split(";").shift();
}

var ua = 'UA-62172390-1';
var clientId = getCookie('_ga');
var tipoHit = 'event';
var pagePath = window.location.pathname;
var pageTitle = document.title;

var xhr = new XMLHttpRequest();
xhr.onreadystatechange = function() {
    if (xhr.readyState == XMLHttpRequest.DONE) {
        console.log(xhr.responseText);
    }
}

var MyAwesomeTagManager = function() {
    var arr = [];
    arr.push = function() {
        console.log("dataLayer actualizado con:", arguments[0]['event']);
        if (arguments[0]['event'] == 'ga') {
            var eventCategory = arguments[0]['eventCategory'];
            var eventAction = arguments[0]['eventAction'];
            var eventLabel = null;
            console.log('Evento ga: | Categoría: ' 
                        + eventCategory
                        + ' | Acción: ' 
                        + eventAction
                       )
            xhr.open("POST", 'http://www.google-analytics.com/collect?v=1&tid=' + ua + '&cid=' + clientId + '&t=' + tipoHit + '&dp=' + pagePath + '&dt=' + pageTitle + '&ec=' + eventCategory + '&ea=' + eventAction + '&el=' + eventLabel + '  ' , true);
            xhr.send('');
        }
        return Array.prototype.push.apply(this, arguments);
    }

    return arr;
};

var dataLayer = new MyAwesomeTagManager;

dataLayer.push({
    'event': 'ga',
    'eventCategory': 'Interacción',
    'eventAction': 'Carrito'
})

In [ ]:
%%javascript
// Quitamos el listener
$('#addToCart').unbind('click');

In [4]:
%%javascript

// Mis variables
function getCookie(name) {
  var value = "; " + document.cookie;
  var parts = value.split("; " + name + "=");
  if (parts.length == 2) return parts.pop().split(";").shift();
}

var ua = 'UA-62172390-1';
var clientId = getCookie('_ga');
var tipoHit = 'event';
var pagePath = window.location.pathname;
var pageTitle = document.title;

var xhr = new XMLHttpRequest();
xhr.onreadystatechange = function() {
    if (xhr.readyState == XMLHttpRequest.DONE) {
        console.log(xhr.responseText);
    }
}


// Mi propio Tag Manager

var MyAwesomeTagManager = function() {
    var arr = [];
    arr.push = function() {
        
        console.log("dataLayer actualizado con:", arguments[0]['event']);
        
        // Un trigger o regla
        if (arguments[0]['event'] == 'ga') {
            var eventCategory = arguments[0]['eventCategory'];
            var eventAction = arguments[0]['eventAction'];
            var eventLabel = null;
            
            console.log('Evento ga: | Categoría: ' 
                        + eventCategory
                        + ' | Acción: ' 
                        + eventAction
                       )
            
            xhr.open("POST", 'http://www.google-analytics.com/collect?v=1&tid=' + ua + '&cid=' + clientId + '&t=' + tipoHit + '&dp=' + pagePath + '&dt=' + pageTitle + '&ec=' + eventCategory + '&ea=' + eventAction + '&el=' + eventLabel + '  ' , true);
            xhr.send(null);
        }
        return Array.prototype.push.apply(this, arguments);
    }

    return arr;
};

var dataLayer = new MyAwesomeTagManager;

$('#addToCart').click(function(){
    dataLayer.push({
        'event': 'ga',
        'eventCategory': 'Interacción',
        'eventAction': 'Carrito'
    })
});

<IPython.core.display.Javascript object>

## ¿Lo vemos todo junto?

[Link](final.ipynb)